A comparison to the below

https://github.com/tirthajyoti/Interactive_Machine_Learning/blob/master/Interactive%20ML-Regression.ipynb

In [1]:
from reactpy import Reactive, Interact, Plot
import numpy as np
import matplotlib.pyplot as plt
%matplotlib widget

import pandas as pd

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LassoCV
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline

In [3]:
r = Reactive(lazy_eval=False)
r.update(get_ipython().user_ns)
get_ipython().user_ns =  r

In [4]:
N_samples = Interact('Samples', {'Low (50 samples)':50,'High (200 samples)':200})
x_min = Interact('Min X', (-5,0,1))
x_max = Interact('Max X', (0,5,1))
noise_mag = Interact('Mag Noise', (0,5,1))
noise_sd = Interact('Std. Noise', (0.1,1,0.1))
noise_mean = Interact('Mean Noise', (-2,2,0.5))

x1= r(lambda x_min, x_max, N_samples: np.linspace(x_min,x_max,N_samples*5))
x = r(lambda x1, N_samples: np.random.choice(x1, size=N_samples))
y = r(lambda x: 2*x-0.6*x**2+0.2*x**3+18*np.sin(x))
y1 = r(lambda x1: 2*x1-0.6*x1**2+0.2*x1**3+18*np.sin(x1))
yp = r(lambda y, noise_mag, noise_mean, noise_sd, N_samples: y+noise_mag*np.random.normal(
    loc=noise_mean,scale=noise_sd,size=N_samples))

def plot(ax,x,yp,x1,y1):
    ax.plot(x1,y1,c='k',lw=2)
    ax.scatter(x,yp,edgecolors='k',c='yellow',s=60)
    ax.grid(True)    

p = Plot(plot)

interactive(children=(Dropdown(description='value', options={'Low (50 samples)': 50, 'High (200 samples)': 200…

interactive(children=(IntSlider(value=-3, description='value', max=0, min=-5), Output()), _dom_classes=('widge…

interactive(children=(IntSlider(value=2, description='value', max=5), Output()), _dom_classes=('widget-interac…

interactive(children=(IntSlider(value=2, description='value', max=5), Output()), _dom_classes=('widget-interac…

interactive(children=(FloatSlider(value=0.5, description='value', max=1.0, min=0.1), Output()), _dom_classes=(…

interactive(children=(FloatSlider(value=0.0, description='value', max=2.0, min=-2.0, step=0.5), Output()), _do…

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [5]:
lasso_eps = 0.01
lasso_nalpha=20
lasso_iter=3000
ridge_alphas = (0.001,0.01,0.1,1)

In [6]:
model_type = Interact('Model', ['Linear regression','LASSO with CV', 'Ridge with CV'])
test_size = Interact('Test Size %', (10,99,5))
degree = Interact('Degree', (1,10,1))

split = r(lambda x,y,test_size: train_test_split(
    x,y,test_size=test_size*0.01,random_state=55))
X_train = r(lambda split: split[0].reshape(-1,1))
X_test  = r(lambda split: split[1].reshape(-1,1))
y_train = r(lambda split: split[2])
y_test  = r(lambda split: split[3])

def get_model(X_train, y_train, model_type, degree, ):
    if (model_type=='Linear regression'):
        model = make_pipeline(PolynomialFeatures(degree,interaction_only=False), 
                          LinearRegression(normalize=True))
    if (model_type=='LASSO with CV'):    
        model = make_pipeline(PolynomialFeatures(degree,interaction_only=False), 
                              LassoCV(eps=lasso_eps,n_alphas=lasso_nalpha,
                                        max_iter=lasso_iter,normalize=True,cv=5))        
    if (model_type=='Ridge with CV'):    
        model = make_pipeline(PolynomialFeatures(degree,interaction_only=False), 
                              RidgeCV(alphas=ridge_alphas,normalize=True,cv=5))
    model.fit(X_train,y_train)
    return model

model = r(get_model)
train_pred  = r(lambda model, X_train: model.predict(X_train))
train_score = r(lambda model, X_train, y_train: model.score(X_train,y_train))
test_pred   = r(lambda model, X_test: model.predict(X_test))
test_score  = r(lambda model, X_test, y_test: model.score(X_test,y_test))
RMSE_test   = r(lambda test_pred, y_test: np.sqrt(np.mean(np.square(test_pred-y_test))))
RMSE_train  = r(lambda train_pred, y_train: np.sqrt(np.mean(np.square(train_pred-y_train))))

fig,(ax1, ax2) = plt.subplots(1, 2, figsize=(10,6))
ax1.set_xlim(-3,3)
ax1.set_ylim(-30,30)
ax2.set_xlim(-3,3)
ax2.set_ylim(-30,30)

def plot1(ax, test_score, X_test, y_test, test_pred):
    ax.set_title("Test set performance\nTest score: %.3f"%(test_score),fontsize=16)
    ax.set_xlabel("X-test",fontsize=13)
    ax.set_ylabel("y-test",fontsize=13)
    ax.scatter(X_test,y_test,edgecolors='k',c='blue',s=60)
    ax.scatter(X_test,test_pred,edgecolors='k',c='yellow',s=60)
    ax.grid(True)
    ax.legend(['Actual test values','Predicted values'])
    
def plot2(ax, train_score, X_train, y_train, train_pred):
    ax.set_title("Training set performance\nTraining score: %.3f"%(train_score),fontsize=16)
    ax.set_xlabel("X-train",fontsize=13)
    ax.set_ylabel("y-train",fontsize=13)
    ax.scatter(X_train,y_train,c='blue')
    ax.scatter(X_train,train_pred,c='yellow')
    ax.grid(True)
    ax.legend(['Actual training values','Fitted values'])
    
p2 = Plot(plot1, ax=ax1)
p3 = Plot(plot2, ax=ax2)


interactive(children=(Dropdown(description='value', options=('Linear regression', 'LASSO with CV', 'Ridge with…

interactive(children=(IntSlider(value=50, description='value', max=99, min=10, step=5), Output()), _dom_classe…

interactive(children=(IntSlider(value=5, description='value', max=10, min=1), Output()), _dom_classes=('widget…

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …